# NTCIR-13 MedWeb - Baseline
Baselineモデル （単語表層による文書分類モデル）

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from typing import *

# ML関連
import sklearn
from sklearn import naive_bayes
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn import model_selection, preprocessing
from sklearn.feature_extraction import text, DictVectorizer
from sklearn import pipeline
from sklearn import metrics
import fasttext

# Parser
from natto import MeCab, MeCabNode
# import Mykytea

# NEologd辞書Path
NEOLOGD = '/usr/lib/mecab/dic/mecab-ipadic-neologd/'

In [3]:
ja_train = pd.read_excel('data/ja_train_20170501.xlsx', sheetname='ja_train')
en_train = pd.read_excel('data/en_train_20170501.xlsx', sheetname='en_train')
posi_nega_columns = ja_train.columns[2:]

In [4]:
# ラベルを0,1へ変換
def sign(s:str) -> int:
    return 1 if s == 'p' else 0

In [5]:
# p, n -> 1, -1
for c in posi_nega_columns:
    ja_train[c] = ja_train[c].apply(sign).astype(np.int)
    en_train[c] = en_train[c].apply(sign).astype(np.int)
ja_train.head()

,ID,Tweet,Influenza,Diarrhea,Hayfever,Cough,Headache,Fever,Runnynose,Cold
0,1ja,風邪を引くと全身がだるくなる。,0,0,0,0,0,0,0,1
1,2ja,花粉症の症状が出てきたのは久し振りだ。,0,0,1,0,0,0,1,0
2,3ja,花粉症のせいでずっと微熱でぼーっとしてる。眠い。,0,0,1,0,0,1,1,0
3,4ja,薬飲んだけど鼻水おさまる気配なし,0,0,0,0,0,0,1,0
4,5ja,ネパールに旅行に行った際に下痢になって大変だったよ。,0,0,0,0,0,0,0,0


In [24]:
ja_train.Tweet.to_csv('data/ja_raw.tsv', sep='\t', index=False)
en_train.Tweet.to_csv('data/en_raw.tsv', sep='\t', index=False)

In [6]:
# MeCab-NEologd, KyTea（※未使用）による単語分割用関数

def parse_func_gen(parser:MeCab):
    def parse_to_morphs(s:str) -> List[Tuple[str, str]]:
        return [tuple(l.split('\t')) for l in parser.parse(s).split('\n')]
    return parse_to_morphs

def parse_func_mecab(parser:MeCab):
    def parse_to_surf(s:str) -> List[str]:
        return  [node.surface for node in parser.parse(s, as_nodes=True) if node.surface]
    return parse_to_surf

# def parse_func_kytea(parser:Mykytea):
#     def parse_to_surf(s:str) -> List[str]:
#         return [r.surface for r in parser.getAllTags(s)]
#     return parse_to_surf

In [7]:
mecab_parser = parse_func_mecab(parser=MeCab("-d {}".format(NEOLOGD)))
ja_train.Tweet_m = ja_train.Tweet.apply(mecab_parser)
ja_train.Tweet_m.head()

0                    [風邪, を, 引く, と, 全身, が, だるく, なる, 。]
1        [花粉症, の, 症状, が, 出, て, き, た, の, は, 久し振り, だ, 。]
2    [花粉症, の, せい, で, ずっと, 微熱, で, ぼーっ, と, し, てる, 。, ...
3                     [薬, 飲ん, だ, けど, 鼻水, おさまる, 気配, なし]
4    [ネパール, に, 旅行, に, 行っ, た, 際, に, 下痢, に, なっ, て, 大変...
Name: Tweet, dtype: object

In [8]:
mecab_parser2 = parse_func_gen(parser=MeCab("-d {}".format(NEOLOGD)))
ja_train.Tweet_mfull = ja_train.Tweet.apply(mecab_parser2)
ja_train.Tweet_mfull.head()

0    [(風邪, 名詞,一般,*,*,*,*,風邪,カゼ,カゼ), (を, 助詞,格助詞,一般,*...
1    [(花粉症, 名詞,固有名詞,一般,*,*,*,花粉症,カフンショウ,カフンショー), (の...
2    [(花粉症, 名詞,固有名詞,一般,*,*,*,花粉症,カフンショウ,カフンショー), (の...
3    [(薬, 名詞,一般,*,*,*,*,薬,クスリ,クスリ), (飲ん, 動詞,自立,*,*,...
4    [(ネパール, 名詞,固有名詞,地域,国,*,*,ネパール,ネパール,ネパール), (に, ...
Name: Tweet, dtype: object

## Baselineモデルの構築
マルチラベル問題を扱える点で，RandomForestをまず利用する．

* 素性
    * 単語表層 BoW，tf-idf
    * 単語Embeddings via FastText

In [9]:
from collections import Counter, defaultdict

In [10]:
def get_counts_of_words(l: List[str]) -> Dict[str, int]:
    return Counter(l)

In [11]:
X = ja_train.Tweet_m.apply(get_counts_of_words).values
Y = ja_train[posi_nega_columns].values.astype(np.float64)

In [12]:
dv = DictVectorizer()
tfidf = text.TfidfTransformer()
X_bow = dv.fit_transform(X)
X_tfidf = tfidf.fit_transform(X_bow)

In [13]:
Xtr, Xval, Ytr, Yval = model_selection.train_test_split(X_tfidf, Y, random_state=12345)

In [14]:
clf = RandomForestClassifier()

In [15]:
clf.fit(Xtr, Ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [16]:
Ypred = clf.predict(Xval)

### マルチラベル問題の評価尺度
[Scikit-learnのドキュメントと解説](http://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)や，[マルチラベル分類メモ](http://d.hatena.ne.jp/jetbead/20140731/1406739777)， [Cross-Validated（Stackoverflowの統計・データサイエンス版）の質問](https://stats.stackexchange.com/questions/233275/multilabel-classification-metrics-on-scikit)を見てみると…

以下のものがマルチラベル問題に使えそう: 
* Accuracy, Precision, Recall, -> F1 (マクロ平均，マイクロ平均の扱いに注意)
* Hamming Loss（ハミング距離，小さい方が良い）

In [17]:
metrics.f1_score(Yval, Ypred, average='macro')

0.76413487189983842

In [18]:
f1_for_cats = metrics.f1_score(Yval, Ypred, average=None).reshape((1, -1))
f1 = pd.DataFrame(f1_for_cats, columns=posi_nega_columns.tolist())
f1

,Influenza,Diarrhea,Hayfever,Cough,Headache,Fever,Runnynose,Cold
0,0.636364,0.829787,0.796296,0.926829,0.796748,0.645669,0.790476,0.690909


In [19]:
metrics.hamming_loss(Yval, Ypred)

0.054947916666666666

In [20]:
print(metrics.classification_report(Yval, Ypred, target_names=posi_nega_columns.tolist()))

             precision    recall  f1-score   support

  Influenza       0.88      0.50      0.64        28
   Diarrhea       0.85      0.81      0.83        48
   Hayfever       0.90      0.72      0.80        60
      Cough       0.98      0.88      0.93        65
   Headache       0.92      0.70      0.80        70
      Fever       0.80      0.54      0.65        76
  Runnynose       0.89      0.71      0.79       117
       Cold       0.93      0.55      0.69        69

avg / total       0.89      0.68      0.77       533



## FastText によるTweet2Vec
さすがに情報の欠損が多い様子．もしくは，Doc2Vecの学習データがWikipediaなことに起因しているかもしれない．

以下のようにあらかじめ各Tweetを300次元の分散表現に落としている．

```shell
 ~/fastText/fasttext print-sentence-vectors /home/public/dataset/nlp/fasttext/wiki.ja.bin < ./ja_raw.tsv > ja_tweet2vec.txt
 ~/fastText/fasttext print-sentence-vectors /home/public/dataset/nlp/fasttext/wiki.en.bin < ./en_raw.tsv > en_tweet2vec.txt
```

中身はこんな感じ: 

```console
風邪を引くと全身がだるくなる。 0.028847 0.080422 -0.06959 -0.050471 -0.0042157 -0.088465 -0.1214 0.012201 -0.032774 -0.00094972 -0.00074691 0.053017 -0.040997 0.0011282 0.076371 0.09389 0.065444 0.043234 0.078271 -0.10056 0.046625 0.10653 0.02981 0.033398 -0.034635 0.060042 -0.025354 0.11413 -0.081827 -0.0057625 -0.045879 -0.030408 -0.033764 -0.033219 -0.082236 0.042674 0.1048 -0.11722 -0.028717 0.15933 0.018732 0.073733 0.042017 -0.065909 0.031493 -0.014718 0.033961 0.073061 0.056836 -0.053601 0.010098 -0.0060847 -0.046822 -0.085515 0.0013303 0.036983 -0.10013 0.0022872 0.096296 -0.014927 0.038335 -0.033982 0.028806 0.0020785 -0.066504 -0.018237 0.044968 0.061053 -0.017108 0.010133 0.045187 0.058719 -0.02964 -0.058711 -0.028329 -0.061745 0.047721 0.01912 0.065648 0.0038917 0.008982 -0.015972 0.044377 -0.0030795 0.040322 0.0996 0.0044982 0.005729 0.017102 -0.09973 0.091635 -0.014136 0.10706 0.035532 -0.070389 -0.0055422 -0.034324 -0.076081 0.040966 0.091609 0.026791 0.088493 0.0022847 0.055996 0.13403 0.12353 -0.056235 -0.0024153 0.10805 0.011953 -0.0018374 0.014711 -0.014808 0.028683 0.096988 -0.094861 0.0282 0.011729 -0.098526 0.021614 -0.0013122 0.023939 0.006392 -0.0087469 -0.043206 -0.10382 -0.013373 0.077495 0.0064959 0.1821 0.1655 0.031694 0.065905 -0.0040029 0.023613 0.016229 -0.161 -0.088657 0.0038904 -0.068251 -0.013907 0.051352 -0.013821 0.049551 -0.0227 0.015025 0.031069 0.015038 0.015902 -0.0099503 0.019562 -0.019366 0.050877 0.081096 -0.051646 0.042915 0.017591 0.020647 -0.0051962 -0.015956 0.001121 -0.0093207 -0.026655 0.047567 0.049813 0.012272 -0.016761 0.023421 0.053676 0.033004 -0.091775 0.006402 0.0091642 -0.096821 -0.038165 -0.012791 0.076949 -0.032315 -0.0013818 -0.088137 0.027177 -0.014497 -0.018414 0.070169 0.0024305 0.050916 -0.036792 -0.038951 -0.012413 -0.02995 0.011732 -0.018853 -0.021139 0.015923 -0.064778 -0.034651 -0.12517 -0.039751 -0.019075 -0.11735 0.10341 -0.051911 0.16146 -0.0087918 0.034713 -0.016807 -0.064736 0.091972 0.01876 0.0058304 -0.032751 0.015891 -0.011265 -0.084611 -0.045655 -0.023582 0.13218 0.010298 -0.021229 0.013076 -0.035317 0.06429 -0.041406 0.091004 -0.091301 -0.027308 0.069603 0.15244 0.048249 0.038457 0.068467 -0.003293 0.081721 -0.10356 -0.054674 -0.019011 0.025771 -0.09025 0.067041 -0.0074499 -0.034265 0.0072296 0.001294 -0.028006 -0.069932 -0.11027 0.0089176 0.026985 -0.051428 0.067124 -0.0098775 0.01961 -0.077425 -0.0025876 0.033908 0.0013121 -0.00065333 -0.062226 0.036521 0.05975 -0.088245 -0.0043476 0.10722 0.022689 -0.021732 -0.016587 0.047553 -0.037922 -0.077617 0.057904 -0.037419 -0.0051642 0.096362 0.0061957 0.0032607 0.039852 0.07092 -0.0065159 -0.045532 0.028812 -0.014446 0.060224 -0.0082398 -0.077256 0.0059774 -0.027994 0.12246 0.077222 0.020402 -0.014792 0.03235 0.0048808 0.040532 0.023096 0.0061202 0.030113 -0.0058276 -0.01372 0.089848 0.0036488
```

In [37]:
X_tw2vec = np.array([list(map(float, l.strip().split()[1:])) for l in open('data/ja_tweet2vec.txt').readlines()])

In [38]:
Xtr, Xval, Ytr, Yval = model_selection.train_test_split(X_tw2vec, Y, random_state=12345)

clf = RandomForestClassifier()
clf.fit(Xtr, Ytr)
Ypred = clf.predict(Xval)
print(metrics.classification_report(Yval, Ypred, target_names=posi_nega_columns.tolist()))

             precision    recall  f1-score   support

  Influenza       0.60      0.11      0.18        28
   Diarrhea       0.00      0.00      0.00        48
   Hayfever       1.00      0.02      0.03        60
      Cough       1.00      0.02      0.03        65
   Headache       0.00      0.00      0.00        70
      Fever       0.67      0.08      0.14        76
  Runnynose       0.56      0.08      0.14       117
       Cold       0.00      0.00      0.00        69

avg / total       0.48      0.04      0.07       533



/opt/miniconda3/envs/jupyterhub/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## モデルを作るなら…

* 機械学習でごり押しするなら…
    * アンサンブルモデル
    * 単語ベクトルを素性としたRNN識別モデル？
        * 学習事例を作るのが大変そう
        
* 素性工学
    * 時制（過去形），伝聞を区別する素性
    * and so on...